In [2]:
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host':'cl-analytics.mwt2.org', 'port':9200}],
                   send_get_body_as = 'POST'
                  )

q = {
  "size": 0,
  "query": {
    "constant_score": {
      "filter": {
        "bool": {
          "must": [
            {"terms": {"computingsite":["TRIUMF_MCORE", "TRIUMF_MCORE_LOMEM"]}},
            {"terms": {"jobstatus": ['finished', 'failed']}},
            {"range": {"endtime": {"from": "2016-11-01T00:00:00",
                                   "to":   "2016-12-01T00:00:00"}}}
          ]
        }
      }
    }
  },
  "aggs": {
    "cpueff": {
      "histogram": {
        "script": {
          "inline": "doc['cpu_eff'].value/6*100",
          "lang": "expression"
        },
        "interval": 10,
        "min_doc_count": 1
      },
      "aggs": {
        "ptype": {
          "terms": {
            "field": "processingtype",
            "size": 50,
            "order": {
              "_term": "asc"
            }
          }
        }
      }
    }
  }
}

index = "jobs_archive_2016-11*,jobs-archive_2016-12*"
res = es.search(index=index, body=q, request_timeout=60)

for cpueff in res['aggregations']['cpueff']['buckets']:
    print cpueff['key'],
    for ptype in cpueff['ptype']['buckets']:
        print ' %s=%d' % (ptype['key'], ptype['doc_count']),
    print


0.0  gangarobot-celpft=196  gangarobot-mcore=46  gangarobot-newmover=163  gangarobot-rcmtest=33  merge=3122  pile=640  recon=50  reprocessing=779  simul=495
10.0  gangarobot-celpft=737  gangarobot-newmover=57  merge=959  pile=317  recon=30  reprocessing=924  simul=309
20.0  gangarobot-celmcore=6  gangarobot-mcore=115  gangarobot-rcmtest=16  merge=1853  pile=327  recon=48  reprocessing=1549  simul=2
30.0  gangarobot-celmcore=1  gangarobot-mcore=21  gangarobot-rcmtest=15  merge=4364  pile=649  recon=26  reprocessing=5151
40.0  gangarobot-celmcore=12  gangarobot-mcore=61  gangarobot-rcmtest=11  merge=2367  pile=1841  recon=1  reprocessing=12058  simul=11  urgent=2
50.0  gangarobot-celmcore=28  gangarobot-mcore=289  gangarobot-rcmtest=19  merge=2320  pile=837  recon=1  reprocessing=9289  simul=10  urgent=165
60.0  gangarobot-celmcore=7  gangarobot-mcore=62  gangarobot-rcmtest=2  merge=3927  pile=325  reprocessing=4625  simul=297  urgent=57
70.0  merge=14161  pile=2948  reprocessing=4539  s